# Manual Annotation and Create target column

In [47]:
import numpy as np

# Load the saved .npz file
num_data = np.load("all_frame_snippets.npz", allow_pickle=True)

# Retrieve the frame snippets and embeddings
frame_snippets = num_data["frame_snippets"]


In [48]:
num_data

In [49]:
frame_snippets[0]

{'name': 'M2BS24P3.wav',
 'start_time': 0.2,
 'end_time': 1.2,
 'embeddings': array([[-4.2187799e-02,  2.4196939e-01,  3.3333007e-02, ...,
         -8.5409455e-02, -2.5977384e-02, -3.6795463e-02],
        [-4.1441415e-02,  2.3888399e-01,  3.1850282e-02, ...,
         -7.1899995e-02, -3.1433553e-02, -3.8360741e-02],
        [-9.4579525e-02, -5.6911781e-03,  1.4942250e-01, ...,
          9.7062998e-02, -1.4417510e-01, -3.9152592e-02],
        ...,
        [-3.2114323e-02,  2.3110186e-01,  5.4990808e-03, ...,
         -8.4011041e-02,  3.6408934e-03, -2.7097853e-02],
        [-3.3829678e-02,  2.2495380e-01,  7.9835281e-03, ...,
         -8.6970754e-02, -4.7619045e-03, -2.4850897e-02],
        [-3.4508813e-02,  2.1230572e-01,  7.7835401e-03, ...,
         -9.1179743e-02,  2.1480254e-04, -2.4080541e-02]], dtype=float32)}

### load stat csv file to make the comparison

In [50]:

import pandas as pd 
df = pd.read_csv(r'/home/aaa470/gsoc/code/stat.csv')

In [51]:
df.head()

,Unnamed: 0,name,cd,gender,break_indices,end_time,series,color,noisy
0,0,F1AS01P1,CD1,F1A,1,0.21,S01,76,0
1,1,F1AS01P1,CD1,F1A,1,0.47,S01,76,0
2,2,F1AS01P1,CD1,F1A,1,0.76,S01,76,0
3,3,F1AS01P1,CD1,F1A,4,1.35,S01,76,1
4,4,F1AS01P1,CD1,F1A,1,1.50,S01,76,0


In [52]:
df.drop('Unnamed: 0' , axis = 1 , inplace = True)

In [53]:
for i in range(0 , len(frame_snippets)):
    start_time = frame_snippets[i]

In [54]:
df

,name,cd,gender,break_indices,end_time,series,color,noisy
0,F1AS01P1,CD1,F1A,1,0.21,S01,76,0
1,F1AS01P1,CD1,F1A,1,0.47,S01,76,0
2,F1AS01P1,CD1,F1A,1,0.76,S01,76,0
3,F1AS01P1,CD1,F1A,4,1.35,S01,76,1
4,F1AS01P1,CD1,F1A,1,1.50,S01,76,0
...,...,...,...,...,...,...,...,...
20483,M2BS04P4,CD2,M2B,1,16.13,S03,76,0
20484,M2BS04P4,CD2,M2B,1,16.40,S03,76,0
20485,M2BS04P4,CD2,M2B,1,16.70,S03,76,0
20486,M2BS04P4,CD2,M2B,1,16.76,S03,76,0


### Convert start_time, end_time, name into dataframe to create a target column

In [55]:
name = []
start_time = []
end_time = []

for i in range(0 , len(frame_snippets)):

    name.append(frame_snippets[i]['name'])
    start_time.append(frame_snippets[i]['start_time'])
    end_time.append(frame_snippets[i]['end_time'])

In [56]:
a = []
for i in name:
    a.append(i.split('.')[0])

In [57]:
data = pd.DataFrame({'name' : a , 'start_time' : start_time , 'end_time' : end_time})

In [58]:
data

,name,start_time,end_time
0,M2BS24P3,0.2,1.2
1,M2BS24P3,1.4,2.4
2,M2BS24P3,2.6,3.6
3,M2BS24P3,3.8,4.8
4,M2BS24P3,5.0,6.0
...,...,...,...
19102,F2BS02P5,27.8,28.8
19103,F2BS02P5,29.0,30.0
19104,F2BS02P5,30.2,31.2
19105,F2BS02P5,31.4,32.4


### Comparison between two dataframes 

In [59]:
unique_names = data['name'].unique()
unique_names2 = df['name'].unique()

for i in unique_names:
    if i in unique_names2:
        x = data[data['name'] == i]
        y = df[df['name'] == i]
        def new_col(col):
            return int(y[(y["end_time"] >= col.start_time) & (y["end_time"] < col.end_time)]["break_indices"].isin([4]).any())

        data["target"] = data.apply(new_col, axis=1)
        continue

data

,name,start_time,end_time,target
0,M2BS24P3,0.2,1.2,0
1,M2BS24P3,1.4,2.4,0
2,M2BS24P3,2.6,3.6,0
3,M2BS24P3,3.8,4.8,1
4,M2BS24P3,5.0,6.0,0
...,...,...,...,...
19102,F2BS02P5,27.8,28.8,1
19103,F2BS02P5,29.0,30.0,0
19104,F2BS02P5,30.2,31.2,0
19105,F2BS02P5,31.4,32.4,1


In [60]:
data.to_csv(r'/home/aaa470/gsoc/code/annotation.csv')